In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import pickle
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [2]:
tf.__version__

'1.8.0'

<h3>Load data</h3>

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
def load_cifar10(data_path):

    train_data = None
    train_labels = []
    test_data = None
    test_labels = None


    for i in range(1, 6):
        data_dict = unpickle(data_path + "data_batch_" + str(i))
        if (i == 1):
            train_data = data_dict[b'data']
        else:
            train_data = np.vstack((train_data, data_dict[b'data']))
        train_labels += data_dict[b'labels']

    test_data_dict = unpickle(data_path + "test_batch")
    test_data = test_data_dict[b'data']
    test_labels = test_data_dict[b'labels']

    train_data = train_data.reshape((50000, 3, 32, 32))
    train_data = np.rollaxis(train_data, 1, 4)
    train_labels = np.array(train_labels)
    
    test_data = test_data.reshape((10000, 3, 32, 32))
    test_data = np.rollaxis(test_data, 1, 4)
    test_labels = np.array(test_labels)
    
    return train_data, train_labels, test_data, test_labels

In [24]:
data_path = "../../cifar-10-batches-py/"
train_data, train_labels, test_data, test_labels = load_cifar10(data_path)

In [25]:
print("train_data.shape:", train_data.shape, "train_labels.shape:", train_labels.shape)
print("test_data.shape:", test_data.shape, "test_labels.shape:", test_labels.shape)

train_data.shape: (50000, 32, 32, 3) train_labels.shape: (50000,)
test_data.shape: (10000, 32, 32, 3) test_labels.shape: (10000,)


<h3>Data preprocessing</h3>

In [26]:
def scale_data(data):
    """ 
    Scale the row pixel intensities to the range [0, 1]
    """
    data = data.astype(np.float32) / 255.0
    return data

train_data = scale_data(train_data)
test_data = scale_data(test_data)

In [13]:
# def one_hot_labels(label_data):
#     """
#     One hot encode the labels
#     """
#     label_data = tf.one_hot(label_data, depth=10)
    
#     return label_data

# train_labels = one_hot_labels(train_labels)
# test_labels = one_hot_labels(test_labels)

In [35]:
# with tf.Session() as sess:
#     print(type(sess.run(train_labels)))
#     print(sess.run(train_labels[:5]))

In [29]:
def one_hot_labels(label_data, num_classes=10):
    """
    One hot encode the labels
    """
    label_data = np.eye(num_classes)[label_data.reshape(-1)]
    
    return label_data

train_labels = one_hot_labels(train_labels)
test_labels = one_hot_labels(test_labels)

In [30]:
train_labels[:5]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

<h3>Train/validation split</h3>

In [36]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_labels, \
                                                      test_size=0.1, \
                                                      random_state=42)

print("X_train.shape:", X_train.shape, "y_train.shape", y_train.shape)
print("X_valid.shape:", X_valid.shape, "y_valid.shape", y_valid.shape)

X_train.shape: (45000, 32, 32, 3) y_train.shape (45000, 10)
X_valid.shape: (5000, 32, 32, 3) y_valid.shape (5000, 10)


<h3>LeNet model</h3>

In [3]:
?tf.layers.conv2d()

In [4]:
?tf.layers.averagepooling2d()

Object `tf.layers.averagepooling2d` not found.


In [ ]:
def LeNet_model(x):
    
    conv1 = tf.layers.conv2d(inputs=x, filters=6, kernel_size=(5, 5), strides=(1, 1), \
                             activation=tf.tanh, kernel_initializer=glorot_uniform_initializer, 
                             padding='valid', name="conv1"
                             )
    avgpool1 = tf.layers.AveragePooling2D(pool_size=[2, 2])